In [2]:
%% P A R s
WDIR                = '~/work/People/Michela/andosols_spatstat';

%% L O A D
load( fullfile(WDIR,'data4models.mat'), 'DATA2' )


In [ ]:
whos

  Name         Size                Bytes  Class    Attributes

  DATA2      865x273            28335006  cell               
  WDIR         1x39                   78  char               
  ans          1x21                   42  char               



In [2]:
version

ans =
9.1.0.441655 (R2016b)


In [ ]:
dir( fullfile(WDIR,'data4models.mat') )


data4models.mat  

